In [1]:
import findspark 
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
import pandas as pd
from datetime import datetime, timedelta,date
import pyspark.sql.functions as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime as dt
import xlsxwriter
from sklearn.linear_model import LinearRegression 
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType, FloatType ,DateType
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.config('spark.executor.memory','4G').appName('recommend').config('spark.driver.cores','4').config('spark.driver.memory','4G').config('spark.driver.maxResultSize','4G').getOrCreate()
df=spark.read.csv('C:/Users/PC/Downloads/rating.csv',header=True,inferSchema=True)
df2=spark.read.csv('C:/Users/PC/Downloads/movie.csv',header=True,inferSchema=True)
df_final=df.join(df2,on='movieId',how='inner')
df_final.show()

+-------+------+------+-------------------+--------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|              genres|
+-------+------+------+-------------------+--------------------+--------------------+
|      2|     1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|Adventure|Childre...|
|     29|     1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|Adventure|Drama|F...|
|     32|     1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|
|     47|     1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|Crime|Mystery|Thr...|
|    112|     1|   3.5|2004-09-10 03:09:00|Rumble in the Bro...|Action|Adventure|...|
|    151|     1|   4.0|2004-09-10 03:08:54|      Rob Roy (1995)|Action|Drama|Roma...|
|    223|     1|   4.0|2005-04-02 23:46:13|       Clerks (1994)|              Comedy|
|    253|     1|   4.0|2005-04-02 23:35:40|Interview w

In [3]:
training,test = df_final.randomSplit([0.8,0.2])
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')
model=als.fit(training)
predictions=model.transform(test)
predictions.show()

+-------+------+------+-------------------+----------------+--------------------+----------+
|movieId|userId|rating|          timestamp|           title|              genres|prediction|
+-------+------+------+-------------------+----------------+--------------------+----------+
|      1|   159|   5.0|1996-12-16 17:47:30|Toy Story (1995)|Adventure|Animati...|  4.559891|
|      1| 11748|   3.5|2012-02-18 19:56:44|Toy Story (1995)|Adventure|Animati...| 3.5113256|
|      1| 11858|   2.0|1997-06-12 08:19:21|Toy Story (1995)|Adventure|Animati...| 3.0455782|
|      1| 11885|   3.0|1996-11-06 12:55:33|Toy Story (1995)|Adventure|Animati...| 3.7539809|
|      1| 11928|   4.5|2011-08-23 22:42:32|Toy Story (1995)|Adventure|Animati...| 4.3567924|
|      1| 23323|   4.0|2014-12-09 23:33:24|Toy Story (1995)|Adventure|Animati...|  3.014052|
|      1| 23408|   4.0|1996-05-27 09:35:04|Toy Story (1995)|Adventure|Animati...| 4.2978706|
|      1| 23421|   4.0|2000-09-06 20:22:37|Toy Story (1995)|Adventure|

In [4]:
# test_id_22 = df_final.filter(df_final['userId']==22).select(['title','userId','movieId'])
# recommendation=model.transform(test_id_22)
# recommendation.orderBy('prediction',ascending=False).show()

#### Top 10 bộ phim được khuyến nghị cho mỗi người dùng

In [5]:
userRecs = model.recommendForAllUsers(10)
userRecs.show()

C:\spark-3.2.1-bin-hadoop2.7\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[{106156, 14.4870...|
|    31|[{86491, 18.82240...|
|    34|[{60356, 11.25679...|
|    53|[{82261, 10.58772...|
|    65|[{93921, 9.337675...|
|    78|[{96255, 17.99517...|
|    81|[{96255, 15.72323...|
|    85|[{81975, 10.57123...|
|   101|[{109953, 9.83567...|
|   108|[{81898, 10.92555...|
|   115|[{89632, 9.027865...|
|   126|[{60356, 10.70423...|
|   133|[{73390, 9.430267...|
|   137|[{126219, 9.69088...|
|   148|[{98126, 11.71515...|
|   155|[{86491, 9.923155...|
|   183|[{82051, 14.03629...|
|   193|[{71482, 19.44056...|
|   210|[{77931, 17.32062...|
|   211|[{84485, 9.688506...|
+------+--------------------+
only showing top 20 rows



#### Top 10 bộ phim được recommend cho người dùng có ID = 22 

In [6]:
users = df_final.filter(df_final['userId']==22)
userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    22|[{73529, 10.55327...|
+------+--------------------+



In [7]:
df_movie=df2.toPandas()
a=list(userSubsetRecs.select('recommendations').toPandas()['recommendations'])
b=list(a[0][i][0] for i in range(10))
list_movie=pd.DataFrame()
list_movie['movieId']=b
list_movie.merge(df_movie,on='movieId',how='inner')

,movieId,title,genres
0,73529,God’s Wedding (As Bodas de Deus) (1999),Comedy
1,81898,"Agony and the Ecstasy of Phil Spector, The (2009)",Documentary
2,82051,Dishonored (1931),Drama|War
3,105084,Grand Masti (2013),Comedy|Drama|Romance
4,120813,Patton Oswalt: My Weakness Is Strong (2009),Comedy
5,89349,"Misérables, Les (1934)",Drama
6,81975,"Statues Also Die (Statues meurent aussi, Les) ...",Documentary
7,88092,Education for Death (1943),Animation|Comedy
8,91517,"Death King, The (Der Todesking) (1990)",Drama|Horror
9,84504,Raja (2003),Drama
